based on notebooks in  
https://github.com/dandi/example-notebooks/tree/master/tutorials/neurodatarehack_2024 

In [185]:
import json
import numpy as np
from dandi.dandiapi import DandiAPIClient
from tqdm.notebook import tqdm
from isodate import parse_duration, Duration
from datetime import datetime
from warnings import simplefilter
simplefilter("ignore")  # Suppress namespace warnings from reading older NWB files

from nwbinspector.tools import get_s3_urls_and_dandi_paths
from pynwb import NWBHDF5IO
import remfile
import h5py

from collections import Counter

import lindi, pynwb

from dandi.dandiapi import DandiAPIClient
import pandas as pd
import pickle

In [198]:
client = DandiAPIClient()
dandisets = list(client.get_dandisets())

# nwb dandisets

In [3]:
nwb_dandisets = []

for dandiset in tqdm(dandisets):
    raw_metadata = dandiset.get_raw_metadata()

    if any(
        data_standard['identifier'] == "RRID:SCR_015242"  # this is the RRID for NWB
        for data_standard in raw_metadata['assetsSummary'].get('dataStandard', [])
    ):
        nwb_dandisets.append(dandiset)
print(f"There are currently {len(nwb_dandisets)} NWB datasets on DANDI!")

  0%|          | 0/697 [00:00<?, ?it/s]

There are currently 420 NWB datasets on DANDI!


# dandiset behavior types 


In [4]:
raw_metadata = dandisets[2].get_raw_metadata()

In [5]:
raw_metadata

{'id': 'DANDI:000005/0.220126.1853',
 'doi': '10.48324/dandi.000005/0.220126.1853',
 'url': 'https://dandiarchive.org/dandiset/000005/0.220126.1853',
 'name': 'Electrophysiology data from thalamic and cortical neurons during somatosensation',
 'about': [{'name': 'dorsal plus ventral thalamus',
   'schemaKey': 'Anatomy',
   'identifier': 'UBERON:0001897'}],
 'access': [{'status': 'dandi:OpenAccess',
   'schemaKey': 'AccessRequirements',
   'contactPoint': {'schemaKey': 'ContactPoint'}}],
 'license': ['spdx:CC-BY-4.0'],
 'version': '0.220126.1853',
 '@context': 'https://raw.githubusercontent.com/dandi/schema/master/releases/0.6.0/context.json',
 'citation': 'Yu, Jianing; Gutnisky, Diego A; Hires, S Andrew; Svoboda, Karel (2022) Electrophysiology data from thalamic and cortical neurons during somatosensation (Version 0.220126.1853) [Data set]. DANDI archive. https://doi.org/10.48324/dandi.000005/0.220126.1853',
 'keywords': [],
 'protocol': [],
 'schemaKey': 'Dandiset',
 'identifier': 'DA

In [56]:

def contains_behavior(data):
    if isinstance(data, dict):
        return any(contains_behavior(v) for v in data.values())
    elif isinstance(data, list):
        return any(contains_behavior(v) for v in data)
    elif isinstance(data, str):
        return 'behavior' in data.lower()
    return False

def find_behavior_keys(data, parent_key=""):
    keys_with_behavior = []

    if isinstance(data, dict):
        for key, value in data.items():
            full_key = f"{parent_key}.{key}" if parent_key else key
            keys_with_behavior.extend(find_behavior_keys(value, full_key))
    elif isinstance(data, list):
        for index, item in enumerate(data):
            full_key = f"{parent_key}[{index}]"
            # keys_with_behavior.extend(find_behavior_keys(item, full_key))
            keys_with_behavior.extend(find_behavior_keys(item, parent_key))
    elif isinstance(data, str):
        if 'behavior' in data.lower():
            keys_with_behavior.append(parent_key)

    return keys_with_behavior


In [104]:
# example
print(find_behavior_keys(raw_metadata))

# query
brbe_nwb_dandisets = []

# contains 'behavior'
for dset in tqdm(nwb_dandisets):
    raw_metadata = dset.get_raw_metadata()

    if contains_behavior(raw_metadata):
        behavior_keys = find_behavior_keys(raw_metadata)
        brbe_nwb_dandisets.append(dset)

print(len(brbe_nwb_dandisets))

['name', 'citation', 'description', 'assetsSummary.approach.name', 'assetsSummary.measurementTechnique.name', 'relatedResource.name']


100%|██████████| 420/420 [00:15<00:00, 26.90it/s]

178


In [459]:
# contains a behavior key

behavior_keys_list = []
behavior_keys_dandisets = []
for dset in tqdm(nwb_dandisets):
    raw_metadata = dset.get_raw_metadata()

    behavior_keys = find_behavior_keys(raw_metadata)
    behavior_keys_list.extend(behavior_keys)
    
    if behavior_keys: 
        approaches = raw_metadata['assetsSummary'].get('approach', [])
        if (
            any('electrophysiological' in a.get('name', '') for a in approaches)
        ):
            exist_ephys = True
        else: 
            exist_ephys = False
        if any('assetssummary' in x.lower() for x in behavior_keys):
            exist_asset = True 
        else:
            exist_asset = False  
        behavior_keys_dandisets.append({
            "dandiset_id": dset.identifier,
            "dandiset": dset,
            "behavior_keys": behavior_keys,
            "asset": exist_asset,
            "ephys": exist_ephys
        })

behavior_keys_dandisets = pd.DataFrame(behavior_keys_dandisets)
# count occurrences of each key
behavior_keys_counter = Counter(behavior_keys_list)
behavior_keys_counter


  0%|          | 0/420 [00:00<?, ?it/s]

Counter({'assetsSummary.approach.name': 122,
         'assetsSummary.measurementTechnique.name': 122,
         'description': 96,
         'assetsSummary.variableMeasured': 91,
         'name': 29,
         'citation': 29,
         'relatedResource.name': 19,
         'keywords': 15,
         'relatedResource.url': 11,
         'studyTarget': 8,
         'contributor.name': 4,
         'about.name': 4,
         'relatedResource.repository': 3,
         'contributor.affiliation.name': 2,
         'acknowledgement': 2,
         'relatedResource.identifier': 1,
         'wasGeneratedBy.description': 1})

In [458]:
behavior_keys_dandisets

,dandiset_id,dandiset,behavior_keys,asset
0,000003,DANDI:000003/0.250624.0409,"[name, citation, description, assetsSummary.ap...",True
1,000004,DANDI:000004/0.220126.1852,"[contributor.affiliation.name, contributor.aff...",False
2,000006,DANDI:000006/0.220126.1855,"[assetsSummary.approach.name, assetsSummary.va...",True
3,000009,DANDI:000009/0.220126.1903,"[assetsSummary.approach.name, assetsSummary.va...",True
4,000010,DANDI:000010/0.220126.1905,"[assetsSummary.approach.name, assetsSummary.va...",True
...,...,...,...,...
173,001515,DANDI:001515/draft,[description],False
174,001530,DANDI:001530/draft,"[assetsSummary.approach.name, assetsSummary.va...",True
175,001533,DANDI:001533/draft,"[assetsSummary.approach.name, assetsSummary.me...",True
176,001539,DANDI:001539/0.250804.1538,"[description, assetsSummary.approach.name, ass...",True


In [460]:
behavior_assets_dandisets = behavior_keys_dandisets[(behavior_keys_dandisets['asset']==True) & 
                                                    (behavior_keys_dandisets['ephys']==True)
                                                    ]
behavior_assets_dandisets

,dandiset_id,dandiset,behavior_keys,asset,ephys
0,000003,DANDI:000003/0.250624.0409,"[name, citation, description, assetsSummary.ap...",True,True
2,000006,DANDI:000006/0.220126.1855,"[assetsSummary.approach.name, assetsSummary.va...",True,True
3,000009,DANDI:000009/0.220126.1903,"[assetsSummary.approach.name, assetsSummary.va...",True,True
4,000010,DANDI:000010/0.220126.1905,"[assetsSummary.approach.name, assetsSummary.va...",True,True
5,000011,DANDI:000011/0.220126.1907,"[assetsSummary.approach.name, assetsSummary.va...",True,True
...,...,...,...,...,...
164,001347,DANDI:001347/0.250528.0702,"[assetsSummary.approach.name, assetsSummary.va...",True,True
167,001371,DANDI:001371/draft,"[description, assetsSummary.approach.name, ass...",True,True
174,001530,DANDI:001530/draft,"[assetsSummary.approach.name, assetsSummary.va...",True,True
175,001533,DANDI:001533/draft,"[assetsSummary.approach.name, assetsSummary.me...",True,True


In [461]:
beh_asset_nwb_dandisets = []

for dset in tqdm(nwb_dandisets):
    raw_metadata = dset.get_raw_metadata()

    approaches = raw_metadata['assetsSummary'].get('approach', [])
    measurement_techniques = raw_metadata['assetsSummary'].get('measurementTechnique', [])
    variables_measured = raw_metadata['assetsSummary'].get('variableMeasured', [])

    if (
        any('behavior' in a.get('name', '').lower() for a in approaches) or
        any('behavior' in m.get('name', '').lower() for m in measurement_techniques) or
        any('behavior' in str(v).lower() for v in variables_measured)
    ):
        beh_asset_nwb_dandisets.append(dset)

print(len(beh_asset_nwb_dandisets))



  0%|          | 0/420 [00:00<?, ?it/s]

122


# read nwb via lindi 

In [464]:
dandiset_id = behavior_assets_dandisets.iloc[1]['dandiset'].get_raw_metadata()['id'].split('/')[0].split(':')[-1]
s3_urls = get_s3_urls_and_dandi_paths(dandiset_id=dandiset_id)
# print(s3_urls)
# print(list(s3_urls.values()))

filepath = list(s3_urls.values())[0]

with DandiAPIClient() as client:
    asset = client.get_dandiset(dandiset_id).get_asset_by_path(filepath)
    s3_url = asset.get_content_url(follow_redirects=1, strip_query=True)
f = lindi.LindiH5pyFile.from_hdf5_file(asset.download_url)
nwb = pynwb.NWBHDF5IO(file=f, mode='r').read()

In [465]:
nwb.objects.values()

dict_values([root pynwb.file.NWBFile at 0x6288612832
Fields:
  acquisition: {
    lick_times <class 'pynwb.behavior.BehavioralEvents'>
  }
  devices: {
    H-129 <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    H-129: 64 <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'pynwb.ecephys.ElectrodesTable'>
  experiment_description: Extracellular electrophysiology recordings performed on mouse anterior lateral motor cortex (ALM) in delay response task. Neural activity from two neuron populations, pyramidal track upper and lower, were characterized, in relation to movement execution.
  experimenter: ['Mike Economo']
  file_create_date: [datetime.datetime(2019, 10, 7, 17, 40, 36, 567483, tzinfo=tzoffset(None, -18000))]
  identifier: anm369962_2017-03-09_0
  institution: Janelia Research Campus
  intervals: {
    trials <class 'pynwb.epoch.TimeIntervals'>
  }
  keywords: <LindiH5pyDataset: /general/keywords>
  related_publications: ['doi:10.1038/s41586-018-0

In [466]:
nwb.experiment_description

np.str_('Extracellular electrophysiology recordings performed on mouse anterior lateral motor cortex (ALM) in delay response task. Neural activity from two neuron populations, pyramidal track upper and lower, were characterized, in relation to movement execution.')

In [288]:
[x for x in nwb.objects.values() if isinstance(x, pynwb.behavior.TimeSeries)]

[lick_right_times pynwb.base.TimeSeries at 0x5487907440
 Fields:
   comments: no comments
   conversion: 1.0
   data: <LindiH5pyDataset: /acquisition/lick_times/lick_right_times/data>
   description: no description
   interval: 1
   offset: 0.0
   resolution: 0.0
   timestamps: <LindiH5pyDataset: /acquisition/lick_times/lick_right_times/timestamps>
   timestamps_unit: seconds
   unit: a.u.,
 lick_left_times pynwb.base.TimeSeries at 0x5487904720
 Fields:
   comments: no comments
   conversion: 1.0
   data: <LindiH5pyDataset: /acquisition/lick_times/lick_left_times/data>
   description: no description
   interval: 1
   offset: 0.0
   resolution: 0.0
   timestamps: <LindiH5pyDataset: /acquisition/lick_times/lick_left_times/timestamps>
   timestamps_unit: seconds
   unit: a.u.]

In [467]:

[x for x in nwb.objects.values() if isinstance(x, pynwb.behavior.SpatialSeries)]

[]

In [290]:
time_array = [x for x in nwb.objects.values() if isinstance(x, pynwb.behavior.TimeSeries)][0].timestamps[:]

# differences between consecutive elements
time_differences = np.diff(time_array)

# Calculate the average difference
average_time_resolution = np.mean(time_differences)

# Print the result
print("Time differences:", time_differences)
print("Average time resolution:", average_time_resolution)

Time differences: [0.144829 0.16667  0.128495 ... 0.140833 0.127666 0.183499]
Average time resolution: 1.1241294529801327


# behavior asset temporal resolution

dandiset name  
get filepaths (for now [0])  
get file, file.download_url (api dandi path)  
stream nwb file  
check if pynwb.behavior class is empty, which  
calculate time resolution
save

In [313]:
import pynwb.behavior
import inspect

def get_non_empty_behavior_classes(nwb):
    """
    Function to find non-empty pynwb.behavior classes in an NWB file.

    Parameters:
        nwb: The NWB file object.

    Returns:
        A dictionary where keys are class names and values are counts of non-empty instances.
    """
    # Get all classes in pynwb.behavior
    behavior_classes = [
        cls for name, cls in inspect.getmembers(pynwb.behavior, inspect.isclass)
        if cls.__module__ == 'pynwb.behavior'
    ]

    # Check which classes are non-empty
    non_empty_classes = {}

    for behavior_class in behavior_classes:
        non_empty_objects = [
            x for x in nwb.objects.values() if isinstance(x, behavior_class)
        ]
        if non_empty_objects:  # If the list is not empty
            non_empty_classes[behavior_class.__name__] = {
                'count': len(non_empty_objects),
                'objects': non_empty_objects,
                }

    return non_empty_classes

In [472]:
behavior_assets_dandisets['filepaths'] = ''

for i, dset in tqdm(behavior_assets_dandisets.iterrows(), total=len(behavior_assets_dandisets)):
    # if i >2:
    #     break
    dandiset_id = dset['dandiset'].get_raw_metadata()['id'].split('/')[0].split(':')[-1]
    with DandiAPIClient() as client:
        c=client.get_dandiset(dandiset_id)
    files = [asset for asset in c.get_assets()]
    behavior_assets_dandisets.at[i, 'filepaths'] = files
    # print(files[0].path)

behavior_assets_dandisets = behavior_assets_dandisets.reset_index()
behavior_assets_dandisets

  0%|          | 0/64 [00:00<?, ?it/s]

In [492]:
files[0].get_content_url(follow_redirects=1, strip_query=True)

'https://dandiarchive.s3.amazonaws.com/blobs/390/a27/390a27ba-13ed-42fb-8709-8fa6bbcca456'

In [681]:
def get_time_info_from_behavior_class(behavior_class, nwb=None):
    # ignore pupil tracking for now

    time_info = {}
    behavior_keys = behavior_class.keys()

    spatial_key_list = ['SpatialSeries', 'Position', 'Eyetracking', 'CompassDirection']
    if behavior_keys:
        for key in behavior_keys:
            try:
                if key == 'BehavioralEvents':
                    try:
                        time_series = behavior_class[key]['objects'][0].time_series
                        time_stamps = list(time_series.values())[0].timestamps[:100]
                        diffs = np.diff(time_stamps)
                        mean_diff = np.mean(diffs)
                        std_diff = np.std(diffs)

                        time_info[key] = {
                            "mean_event_diff": np.round(mean_diff, 6),
                            "std_event_diff": np.round(std_diff, 6)
                        }
                    except Exception as e:
                        time_info[key] = e
                
                elif key == 'BehavioralTimeSeries':
                    obj_time_infos = {}
                    for i in range(len(behavior_class[key]['objects'])):
                        obj = behavior_class[key]['objects'][i]
                        try:
                            # all = [x for x in nwb.objects.values() if isinstance(x, pynwb.behavior.TimeSeries)]
                            # time_stamps = all[0].timestamps[:1000]
                            list_dicts = list(obj.time_series.values())
                            print(len(list_dicts))
                            for j in range(len(list_dicts)):
                                ts = list_dicts[j]
                                print(ts)
                                starting_time = ts.starting_time
                                starting_time_unit = ts.starting_time_unit
                                rate = ts.rate

                                time_stamps = ts.timestamps[:500]
                                if time_stamps is not None:
                                    diffs = np.diff(time_stamps)
                                    mean_diff = np.mean(diffs)
                                    std_diff = np.std(diffs)
                                else:
                                    diffs = None
                                    mean_diff = None
                                    std_diff = None

                                obj_time_infos[obj.name] = {
                                    'name': obj.name,
                                    'rate': rate,
                                    'starting_time': starting_time,
                                    'starting_time_unit': starting_time_unit,
                                    "mean_diff": np.round(mean_diff, 6),
                                    "std_diff": np.round(std_diff, 6)
                                }
                        except Exception as e:
                            obj_time_infos[obj.name] = e
                    time_info[key] = obj_time_infos
                    
                elif key in spatial_key_list:
                    obj_time_infos = {}
                    for i in range(len(behavior_class[key]['objects'])):
                        obj = behavior_class[key]['objects'][i]
                        try:
                            # all = [x for x in nwb.objects.values() if isinstance(x, pynwb.behavior.TimeSeries)]
                            # return all 
                            starting_time = obj.starting_time
                            starting_time_unit = obj.starting_time_unit
                            rate = obj.rate
                            
                            time_stamps = obj.timestamps[:500]
                            diffs = np.diff(time_stamps)
                            mean_diff = np.mean(diffs)
                            std_diff = np.std(diffs)

                            obj_time_infos[obj.name] = {
                                'name': obj.name,
                                'rate': rate,
                                'starting_time': starting_time,
                                'starting_time_unit': starting_time_unit,
                                "mean_diff": np.round(mean_diff, 6),
                                "std_diff": np.round(std_diff, 6)
                            }
                            
                            if rate is  None:
                                rate = 'no rate, check timestamps'
                              
                            
                            
                        except Exception as e:
                            obj_time_infos[obj.name] = e
                    time_info[key] = obj_time_infos
            except Exception as e:
                time_info[key] = Exception
                    

    return time_info

In [ ]:
# behavior_assets_dandisets['experiment_description'] = ''
# behavior_assets_dandisets['behavior_class'] = ''
# behavior_assets_dandisets['time_info'] = ''


In [634]:
nwb

AttributeError: 'str' object has no attribute 'get_storage_size'

root pynwb.file.NWBFile at 0x14377444368
Fields:
  experiment_description: regional and Yeo-network ketamine change in response to eyepuff, with key clusters
  experimenter: ['Kauvar, Isaac' 'Richman, Ethan' 'Liu, Tony']
  file_create_date: [datetime.datetime(2025, 2, 23, 19, 13, 27, 945390, tzinfo=tzoffset(None, -18000))]
  identifier: kauvar-richman-liu-fig3
  institution: Stanford University
  keywords: <LindiH5pyDataset: /general/keywords>
  lab: Karl Deisseroth Lab
  scratch: {
    ACC_change <class 'pynwb.core.ScratchData'>,
    ACC_contour <class 'pynwb.core.ScratchData'>,
    AG_change <class 'pynwb.core.ScratchData'>,
    AG_contour <class 'pynwb.core.ScratchData'>,
    AMY_change <class 'pynwb.core.ScratchData'>,
    AMY_contour <class 'pynwb.core.ScratchData'>,
    BG_change <class 'pynwb.core.ScratchData'>,
    BG_contour <class 'pynwb.core.ScratchData'>,
    CS_change <class 'pynwb.core.ScratchData'>,
    CS_contour <class 'pynwb.core.ScratchData'>,
    Default_change <cla

In [621]:
problems = []

for i, dset in tqdm(behavior_assets_dandisets.iterrows(), total=len(behavior_assets_dandisets)):
    try:

        dandiset_id = dset['dandiset_id']
        first_filepath = dset['filepaths'][0].path

        with DandiAPIClient() as client:
            file=client.get_dandiset(dandiset_id).get_asset_by_path(first_filepath)
            lindi_url = file.download_url
        f = lindi.LindiH5pyFile.from_hdf5_file(lindi_url)
        nwb = pynwb.NWBHDF5IO(file=f, mode='r').read()

        behavior_classes = get_non_empty_behavior_classes(nwb)
        behavior_assets_dandisets.at[i, 'experiment_description'] = nwb.experiment_description
        behavior_assets_dandisets.at[i, 'behavior_class'] = behavior_classes
        
        time_info = get_time_info_from_behavior_class(behavior_classes)
        behavior_assets_dandisets.at[i, 'time_info'] = time_info
    except Exception as e:
        problems.append((i, dandiset_id, dset['dandiset'], e))



  0%|          | 0/64 [00:00<?, ?it/s]

position_sensor1
position_sensor0
Position
position
eye_position
Position
R_Wrist
R_Shoulder
R_Elbow
R_Ear
Nose
L_Wrist
L_Shoulder
L_Elbow
L_Ear
SubjectPosition
RedLED
BlueLED
RedLED
BlueLED
SubjectPosition
SpatialSeries
Position
Hand
Eye
Cursor
position
series_4
series_3
series_2
series_1
series_0
Position
Hand
Eye
DecodePos
Cursor
Position
tube_top_rightCamera
tube_top_leftCamera
tube_bottom_rightCamera
tube_bottom_leftCamera
tongue_end_r_rightCamera
tongue_end_r_leftCamera
tongue_end_l_rightCamera
tongue_end_l_leftCamera
tail_start_bodyCamera
pupil_top_r_rightCamera
pupil_top_r_leftCamera
pupil_right_r_rightCamera
pupil_right_r_leftCamera
pupil_left_r_rightCamera
pupil_left_r_leftCamera
pupil_bottom_r_rightCamera
pupil_bottom_r_leftCamera
paw_r_rightCamera
paw_r_leftCamera
paw_l_rightCamera
paw_l_leftCamera
nose_tip_rightCamera
nose_tip_leftCamera
Position
SpatialSeries
allocentric_frame_tracking
linear_task
position
error_per_marker
orientation
direction
position
animal_position
he

In [622]:
behavior_assets_dandisets.to_csv('./behavior_assets_dandisets.csv')

In [623]:
behavior_assets_dandisets_time_info = behavior_assets_dandisets[['dandiset_id', 'dandiset', 'behavior_class', 'time_info']]
behavior_assets_dandisets_time_info.to_csv('./behavior_assets_dandisets_time_info.csv')

### remfile

In [529]:
# remfile
s3link = files[0].get_content_url(follow_redirects=1, strip_query=True)
print(s3link)
cache_dirname = './tmp/remfile_cache'
# disk_cache = remfile.DiskCache(cache_dirname)
rem_file = remfile.File(s3link, )
h5py_file = h5py.File(rem_file, 'r')
io = NWBHDF5IO(file=h5py_file)
nwbfile = io.read()

https://dandiarchive.s3.amazonaws.com/blobs/390/a27/390a27ba-13ed-42fb-8709-8fa6bbcca456


## spatial series

In [610]:
data = behavior_assets_dandisets.iloc[11]
data

index                                                                    17
dandiset_id                                                          000050
dandiset                                                 DANDI:000050/draft
behavior_keys             [assetsSummary.approach.name, assetsSummary.va...
asset                                                                  True
ephys                                                                  True
experiment_description                                                 None
behavior_class            {'BehavioralTimeSeries': {'count': 2, 'objects...
time_info                 {'BehavioralTimeSeries': {'mean_diff': 0.03316...
filepaths                 [DANDI:assets/f3de94e9-6af4-4169-b911-1e7028ca...
Name: 11, dtype: object

In [611]:
data['behavior_class']

{'BehavioralTimeSeries': {'count': 2,
  'objects': [RunningBehavior pynwb.behavior.BehavioralTimeSeries at 0x13850469520
   Fields:
     time_series: {
       running_speed <class 'pynwb.base.TimeSeries'>
     },
   EyeBehavior pynwb.behavior.BehavioralTimeSeries at 0x13850469200
   Fields:
     time_series: {
       eye_area <class 'pynwb.base.TimeSeries'>,
       pupil_area <class 'pynwb.base.TimeSeries'>,
       screen_coordinates_spherical <class 'pynwb.base.TimeSeries'>
     }]}}

In [612]:
data['time_info']

{'BehavioralTimeSeries': {'mean_diff': np.float64(0.033168),
  'std_diff': np.float64(4e-06)}}

In [620]:
get_time_info_from_behavior_class(data['behavior_class'])


{'BehavioralTimeSeries': {'RunningBehavior': {'name': 'RunningBehavior',
   'mean_diff': np.float64(0.033168),
   'std_diff': np.float64(4e-06)},
  'EyeBehavior': {'name': 'EyeBehavior',
   'mean_diff': np.float64(0.033168),
   'std_diff': np.float64(4e-06)}}}

In [624]:
behavior_assets_dandisets.columns

Index(['index', 'dandiset_id', 'dandiset', 'behavior_keys', 'asset', 'ephys',
       'experiment_description', 'behavior_class', 'time_info', 'filepaths'],
      dtype='object')

# time info

In [641]:
df_time_info = behavior_assets_dandisets.copy()
df_time_info.head(10)

,index,dandiset_id,dandiset,behavior_keys,asset,ephys,experiment_description,behavior_class,time_info,filepaths
0,0,000003,DANDI:000003/0.250624.0409,"[name, citation, description, assetsSummary.ap...",True,True,None,"{'SpatialSeries': {'count': 2, 'objects': [pos...",{'SpatialSeries': {'position_sensor1': 'NoneTy...,[DANDI:assets/5e9e92e1-f044-4aa0-ab47-1cfcb889...
1,2,000006,DANDI:000006/0.220126.1855,"[assetsSummary.approach.name, assetsSummary.va...",True,True,Extracellular electrophysiology recordings per...,"{'BehavioralEvents': {'count': 1, 'objects': [...",{'BehavioralEvents': {'mean_event_diff': 0.981...,[DANDI:assets/a5ad932b-b893-4522-b989-8f406d78...
2,3,000009,DANDI:000009/0.220126.1903,"[assetsSummary.approach.name, assetsSummary.va...",True,True,,{},{},[DANDI:assets/250ea757-e6a9-4520-99b5-f2efd5e3...
3,4,000010,DANDI:000010/0.220126.1905,"[assetsSummary.approach.name, assetsSummary.va...",True,True,Extracellular electrophysiology recordings wit...,"{'BehavioralEvents': {'count': 1, 'objects': [...",{'BehavioralEvents': {'mean_event_diff': 12.00...,[DANDI:assets/6b3b38b9-0736-46a4-a348-b00af509...
4,5,000011,DANDI:000011/0.220126.1907,"[assetsSummary.approach.name, assetsSummary.va...",True,True,Extracellular electrophysiology recordings wit...,"{'BehavioralEvents': {'count': 1, 'objects': [...",{'BehavioralEvents': list index out of range},[DANDI:assets/88dd3ee7-a37a-44b1-bb64-89855040...
5,6,000013,DANDI:000013/0.220126.2143,"[assetsSummary.approach.name, assetsSummary.va...",True,True,Intracellular and extracellular electrophysiol...,"{'BehavioralTimeSeries': {'count': 1, 'objects...",{'BehavioralTimeSeries': {'behavior': {'name':...,[DANDI:assets/cbcf1d6d-7f64-4d1f-8692-75e09e17...
6,9,000017,DANDI:000017/0.240329.1926,"[assetsSummary.approach.name, assetsSummary.va...",True,True,Large-scale Neuropixels recordings across brai...,"{'BehavioralEpochs': {'count': 1, 'objects': [...",{'BehavioralEvents': {'mean_event_diff': 0.582...,[DANDI:assets/2c6984f5-5fd7-4ccb-8f05-1961df65...
7,10,000029,DANDI:000029/0.231017.2004,"[assetsSummary.approach.name, assetsSummary.va...",True,True,Single unit recordings from chronically implan...,{},{},[DANDI:assets/7a3d889c-b513-40d6-b3e9-74ccb24e...
8,13,000039,DANDI:000039/0.230223.1216,"[assetsSummary.approach.name, assetsSummary.va...",True,True,None,"{'BehavioralTimeSeries': {'count': 2, 'objects...",{'BehavioralTimeSeries': {'RunningBehavior': {...,[DANDI:assets/f2ca9a62-7034-4b58-892b-8a3cbf32...
9,15,000045,DANDI:000045/0.211209.1413,"[name, citation, assetsSummary.approach.name, ...",True,True,ibl_neuropixel_brainwide_01,"{'BehavioralTimeSeries': {'count': 1, 'objects...",{'BehavioralTimeSeries': {'BehavioralTimeSerie...,[DANDI:assets/b7855fa3-b6ee-4545-86b4-b4413095...


In [643]:
a = df_time_info.iloc[6]
a['experiment_description']

np.str_('Large-scale Neuropixels recordings across brain regions of mice during a head-fixed visual discrimination task. ')

In [644]:
a['behavior_class']

{'BehavioralEpochs': {'count': 1,
  'objects': [BehavioralEpochs pynwb.behavior.BehavioralEpochs at 0x13832088592
   Fields:
     interval_series: {
       wheel_moves <class 'pynwb.misc.IntervalSeries'>
     }]},
 'BehavioralEvents': {'count': 1,
  'objects': [BehavioralEvents pynwb.behavior.BehavioralEvents at 0x5738999440
   Fields:
     time_series: {
       lick_times <class 'pynwb.base.TimeSeries'>
     }]},
 'BehavioralTimeSeries': {'count': 1,
  'objects': [BehavioralTimeSeries pynwb.behavior.BehavioralTimeSeries at 0x13832088912
   Fields:
     time_series: {
       face_motion_energy <class 'pynwb.base.TimeSeries'>
     }]},
 'PupilTracking': {'count': 1,
  'objects': [PupilTracking pynwb.behavior.PupilTracking at 0x13832089232
   Fields:
     time_series: {
       eye_area <class 'pynwb.base.TimeSeries'>,
       eye_xy_positions <class 'pynwb.base.TimeSeries'>
     }]}}

In [645]:
a['time_info']

{'BehavioralEvents': {'mean_event_diff': np.float64(0.582899),
  'std_event_diff': np.float64(1.258091)},
 'BehavioralTimeSeries': {'BehavioralTimeSeries': TypeError("'NoneType' object is not subscriptable")}}

In [649]:
print(a['behavior_class']['BehavioralTimeSeries']['objects'][0].time_series)

{'face_motion_energy': face_motion_energy pynwb.base.TimeSeries at 0x5738990800
Fields:
  comments: The integrated motion energy across the whole frame, i.e. sum( (thisFrame-lastFrame)^2 ). Some smoothing is applied before this operation.
  conversion: 1.0
  data: <LindiH5pyDataset: /processing/behavior/BehavioralTimeSeries/face_motion_energy/data>
  description: Features extracted from the video of the frontal aspect of the subject, including the subject's face and forearms.
  offset: 0.0
  rate: 0.02524557875164268
  resolution: -1.0
  starting_time: 18.115412920074125
  starting_time_unit: seconds
  unit: arb. unit
}


In [661]:
a['behavior_class']['BehavioralTimeSeries']['objects'][0].time_series

{'face_motion_energy': face_motion_energy pynwb.base.TimeSeries at 0x5738990800
 Fields:
   comments: The integrated motion energy across the whole frame, i.e. sum( (thisFrame-lastFrame)^2 ). Some smoothing is applied before this operation.
   conversion: 1.0
   data: <LindiH5pyDataset: /processing/behavior/BehavioralTimeSeries/face_motion_energy/data>
   description: Features extracted from the video of the frontal aspect of the subject, including the subject's face and forearms.
   offset: 0.0
   rate: 0.02524557875164268
   resolution: -1.0
   starting_time: 18.115412920074125
   starting_time_unit: seconds
   unit: arb. unit}

In [682]:
get_time_info_from_behavior_class(a['behavior_class'])

1
face_motion_energy pynwb.base.TimeSeries at 0x5738990800
Fields:
  comments: The integrated motion energy across the whole frame, i.e. sum( (thisFrame-lastFrame)^2 ). Some smoothing is applied before this operation.
  conversion: 1.0
  data: <LindiH5pyDataset: /processing/behavior/BehavioralTimeSeries/face_motion_energy/data>
  description: Features extracted from the video of the frontal aspect of the subject, including the subject's face and forearms.
  offset: 0.0
  rate: 0.02524557875164268
  resolution: -1.0
  starting_time: 18.115412920074125
  starting_time_unit: seconds
  unit: arb. unit



{'BehavioralEvents': {'mean_event_diff': np.float64(0.582899),
  'std_event_diff': np.float64(1.258091)},
 'BehavioralTimeSeries': {'BehavioralTimeSeries': TypeError("'NoneType' object is not subscriptable")}}

In [678]:
obj

{'BehavioralEvents': {'mean_event_diff': np.float64(0.582899),
  'std_event_diff': np.float64(1.258091)},
 'BehavioralTimeSeries': {'BehavioralTimeSeries': TypeError("'NoneType' object is not subscriptable")}}